# Lab 06-1. Softmax Classifier

In [ ]:
import numpy as np

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

In [ ]:
tf.set_random_seed(777)  # for reproducibility

In [ ]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
# one-hot encoding [a(0), b(1), c(2)] classification
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

In [ ]:
X = tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, 3])
nb_classes = 3 # label의 갯수

W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

In [ ]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

# Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1)) # Y는 one-hot encoding 된 값

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [ ]:
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2001):
            _, cost_val = sess.run([optimizer, cost], feed_dict={X: x_data, Y: y_data})

            if step % 200 == 0:
                print(step, cost_val)
    print('--------------')
    # Testing & One-hot encoding
    a = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9]]})
    print(a, sess.run(tf.argmax(a, 1)))

    print('--------------')
    b = sess.run(hypothesis, feed_dict={X: [[1, 3, 4, 3]]})
    print(b, sess.run(tf.argmax(b, 1)))

    print('--------------')
    c = sess.run(hypothesis, feed_dict={X: [[1, 1, 0, 1]]})
    print(c, sess.run(tf.argmax(c, 1)))

    print('--------------')
    all = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]]})
    print(all, sess.run(tf.argmax(all, 1)))


0 6.926112
200 0.60050154
400 0.47295827
600 0.37342954
800 0.28018364
1000 0.23280516
1200 0.21065347
1400 0.19229901
1600 0.17682335
1800 0.16359559
2000 0.15216161
--------------
[[1.3890535e-03 9.9860185e-01 9.0613430e-06]] [1]
--------------
[[0.9311918  0.06290229 0.00590592]] [0]
--------------
[[1.2732840e-08 3.3411419e-04 9.9966586e-01]] [2]
--------------
[[1.3890523e-03 9.9860185e-01 9.0613348e-06]
 [9.3119180e-01 6.2902264e-02 5.9059248e-03]
 [1.2732840e-08 3.3411419e-04 9.9966586e-01]] [1 0 2]


# Lab 06-2. Softmax Classifier for Zoo

In [ ]:
tf.set_random_seed(777)  # for reproducibility

In [ ]:
# colab 사용시
path=''

In [ ]:
# Predicting animal type based on various features
xy = np.loadtxt(path + 'data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape, x_data.dtype, y_data.dtype)

(101, 16) (101, 1) float32 float32


In [ ]:
nb_classes = 7  # 0 ~ 6

X = tf.placeholder("float", [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

In [ ]:
Y_one_hot = tf.one_hot(Y, nb_classes)  # one-hot encoding
print("one_hot:", Y_one_hot) # rank(차원)이 +1 된다
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print("reshape one_hot:", Y_one_hot)

one_hot: Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)
reshape one_hot: Tensor("Reshape:0", shape=(?, 7), dtype=float32)


In [ ]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                                 labels=tf.stop_gradient([Y_one_hot])))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1) # (logits, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1)) # tf.argmax(Y_one_hot, 1) = y_data
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2001):
        _, cost_val, acc_val = sess.run([optimizer, cost, accuracy], feed_dict={X: x_data, Y: y_data})
                                        
        if step % 100 == 0:
            print("Step: {:5}\tCost(Loss): {:.3f}\tAcc: {:.2%}".format(step, cost_val, acc_val))

    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

Step:     0	Cost(Loss): 6.821	Acc: 3.96%
Step:   100	Cost(Loss): 0.812	Acc: 75.25%
Step:   200	Cost(Loss): 0.464	Acc: 87.13%
Step:   300	Cost(Loss): 0.332	Acc: 88.12%
Step:   400	Cost(Loss): 0.259	Acc: 93.07%
Step:   500	Cost(Loss): 0.212	Acc: 95.05%
Step:   600	Cost(Loss): 0.180	Acc: 97.03%
Step:   700	Cost(Loss): 0.156	Acc: 98.02%
Step:   800	Cost(Loss): 0.138	Acc: 99.01%
Step:   900	Cost(Loss): 0.124	Acc: 99.01%
Step:  1000	Cost(Loss): 0.112	Acc: 99.01%
Step:  1100	Cost(Loss): 0.102	Acc: 99.01%
Step:  1200	Cost(Loss): 0.094	Acc: 99.01%
Step:  1300	Cost(Loss): 0.087	Acc: 99.01%
Step:  1400	Cost(Loss): 0.081	Acc: 99.01%
Step:  1500	Cost(Loss): 0.076	Acc: 100.00%
Step:  1600	Cost(Loss): 0.071	Acc: 100.00%
Step:  1700	Cost(Loss): 0.067	Acc: 100.00%
Step:  1800	Cost(Loss): 0.063	Acc: 100.00%
Step:  1900	Cost(Loss): 0.060	Acc: 100.00%
Step:  2000	Cost(Loss): 0.057	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0